## DIA 018: Evaluacion Avanzada y Preparacion para el Despliegue del Modelo

Evaluación Avanzada del Modelo:

Análisis de Error Detallado: Identificar y analizar las confusiones más comunes del modelo.
Métricas Avanzadas: Calcular métricas como Precision, Recall y F1-Score para cada clase.
Curva ROC y AUC: Generar la Curva ROC y calcular el Área Bajo la Curva para evaluar la capacidad de discriminación del modelo.
Preparación para el Despliegue del Modelo:

Exportar el Modelo para Despliegue: Guardar el modelo en formatos adecuados para diferentes plataformas (por ejemplo, TensorFlow SavedModel, TensorFlow Lite).
Implementar una API de Predicción Simple: Crear una API RESTful utilizando Flask para servir el modelo y realizar predicciones en tiempo real.
Integrar el Modelo en una Aplicación Web Básica: Desarrollar una interfaz web sencilla que permita a los usuarios subir imágenes y obtener predicciones del modelo.

In [2]:
# 1. Importación de Librerías Adicionales
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
import datetime  # Para timestamp en TensorBoard
from flask import Flask, request, jsonify, render_template
from PIL import Image
import io
import pandas as pd


In [3]:
# 2. Evaluación Avanzada del Modelo

# 2.1. Análisis de Error Detallado

# Asumiendo que y_test y clases_predichas_ft ya están definidos y que val_ds es el conjunto de validación

# Imprimir el reporte de clasificación
print("Reporte de Clasificación después del Fine-Tuning Optimizado:\n")
print(classification_report(y_test, clases_predichas_ft))

# Generar la matriz de confusión
cm_ft = confusion_matrix(y_test, clases_predichas_ft)

# Visualizar la matriz de confusión
plt.figure(figsize=(10,8))
sns.heatmap(cm_ft, annot=True, fmt='d', cmap='Greens', 
            xticklabels=np.unique(y_train), 
            yticklabels=np.unique(y_train))
plt.xlabel('Predicción')
plt.ylabel('Realidad')
plt.title('Matriz de Confusión después del Fine-Tuning Optimizado')
plt.show()

# Identificar las confusiones más frecuentes
# Convertir la matriz de confusión en un DataFrame para un análisis más fácil
cm_df = pd.DataFrame(cm_ft, index=np.unique(y_train), columns=np.unique(y_train))

# Eliminar la diagonal (correctas clasificaciones) para enfocarse en los errores
cm_df_no_diag = cm_df.copy()
np.fill_diagonal(cm_df_no_diag.values, 0)

# Encontrar las 5 confusiones más frecuentes
confusiones = cm_df_no_diag.unstack().sort_values(ascending=False)
top_confusiones = confusiones[confusiones > 0].head(5)
print("\nTop 5 Confusiones Más Frecuentes:")
print(top_confusiones)


Reporte de Clasificación después del Fine-Tuning Optimizado:



NameError: name 'y_test' is not defined

In [ ]:
# 2.2. Cálculo de Métricas Avanzadas

# Ya calculado en el classification_report
# Para cada clase:
# Precision: Precisión de la clase
# Recall: Sensibilidad de la clase
# F1-Score: Media armónica de Precision y Recall

# Estas métricas son útiles para entender el rendimiento del modelo en cada clase individualmente


In [ ]:
# 2.3. Generación de Curva ROC y Cálculo de AUC

from sklearn.preprocessing import label_binarize

# Binarizar las etiquetas para ROC
y_test_binarized = label_binarize(y_test, classes=np.unique(y_train))
n_classes = y_test_binarized.shape[1]

# Obtener las predicciones en formato binarizado
predicciones_binarizadas = label_binarize(clases_predichas_ft, classes=np.unique(y_train))

# Calcular las curvas ROC y AUC para cada clase
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], predicciones_ft[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Graficar la Curva ROC para cada clase
plt.figure(figsize=(10,8))
colors = plt.cm.get_cmap('tab10').colors
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i],
             lw=2, label=f'Clase {i} (AUC = {roc_auc[i]:0.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC por Clase')
plt.legend(loc="lower right")
plt.show()


In [ ]:
# 3.1. Exportar el Modelo en Formatos Adecuados

# Guardar el modelo en formato TensorFlow SavedModel
modelo_optimizado.save('saved_model_mnist_final')

# Convertir y guardar el modelo en formato TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model_mnist_final')
tflite_model = converter.convert()

# Guardar el modelo TensorFlow Lite en un archivo
with open('model_mnist.tflite', 'wb') as f:
    f.write(tflite_model)
print("Modelos exportados exitosamente en formatos SavedModel y TensorFlow Lite.")


In [ ]:
# 3.2. Implementar una API de Predicción Simple con Flask

from flask import Flask, request, jsonify
import tensorflow as tf
from PIL import Image
import numpy as np
import io

app = Flask(__name__)

# Cargar el modelo TensorFlow SavedModel
modelo_api = tf.keras.models.load_model('saved_model_mnist_final')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No se encontró el archivo en la solicitud.'}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No se seleccionó ningún archivo.'}), 400
    try:
        # Leer la imagen
        img = Image.open(io.BytesIO(file.read())).convert('L')  # Convertir a escala de grises
        img = img.resize((28, 28))  # Redimensionar a 28x28
        img_array = np.array(img).astype('float32') / 255.0  # Normalizar
        img_array = img_array.reshape((1, 28, 28, 1))  # Añadir dimensión de batch
        
        # Preprocesar para el modelo (redimensionar a 224x224 y convertir a RGB)
        img_resized = tf.image.resize(img_array, [224, 224])
        img_rgb = tf.image.grayscale_to_rgb(img_resized)
        
        # Realizar la predicción
        pred = modelo_api.predict(img_rgb)
        pred_class = np.argmax(pred, axis=1)[0]
        pred_prob = float(np.max(pred))
        
        return jsonify({'prediccion': int(pred_class), 'probabilidad': pred_prob})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Ejecutar la aplicación Flask
if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# 3.3. Desarrollar una Aplicación Web Básica para Interactuar con el Modelo

from flask import Flask, render_template, request, jsonify
import tensorflow as tf
from PIL import Image
import numpy as np
import io

app = Flask(__name__)

# Cargar el modelo optimizado
modelo_web = tf.keras.models.load_model('modelo_cnn_transfer_learning_mnist_final_optimizado.h5')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict_web():
    if 'file' not in request.files:
        return jsonify({'error': 'No se encontró el archivo en la solicitud.'}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No se seleccionó ningún archivo.'}), 400
    try:
        # Leer la imagen
        img = Image.open(io.BytesIO(file.read())).convert('L')  # Convertir a escala de grises
        img = img.resize((28, 28))  # Redimensionar a 28x28
        img_array = np.array(img).astype('float32') / 255.0  # Normalizar
        img_array = img_array.reshape((1, 28, 28, 1))  # Añadir dimensión de batch
        
        # Preprocesar para el modelo (redimensionar a 224x224 y convertir a RGB)
        img_resized = tf.image.resize(img_array, [224, 224])
        img_rgb = tf.image.grayscale_to_rgb(img_resized)
        
        # Realizar la predicción
        pred = modelo_web.predict(img_rgb)
        pred_class = np.argmax(pred, axis=1)[0]
        pred_prob = float(np.max(pred))
        
        return jsonify({'prediccion': int(pred_class), 'probabilidad': pred_prob})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Ejecutar la aplicación Flask
if __name__ == '__main__':
    app.run(debug=True)
